# Open AI Function Calling

In [ ]:
%pip install openai

In [ ]:
import os.path
import requests

# Download einer Beispiel-Speisekarte, welche bereits in strukturiertem Python-Code vorliegt.
# Könnte man später durch einen Datei- oder Webseitenimport ersetzen.

MENU_DEMO_URL = "https://raw.githubusercontent.com/TAS-AG/TAS.OpenAI.Workshop/refs/heads/main/data/speisekarte-pizza-sample.py"
MENU_MICELLOS_URL = "https://github.com/TAS-AG/TAS.OpenAI.Workshop/blob/main/data/speisekarte_micellos_leipzig.pdf"
MENU_ATRIUM_URL = "https://github.com/TAS-AG/TAS.OpenAI.Workshop/blob/main/data/bistro-im-atrium-wochenkarte.html"

download_urls = [MENU_DEMO_URL, MENU_MICELLOS_URL, MENU_ATRIUM_URL]

for url in download_urls:
    response = requests.get(url)
    filename = os.path.basename(url)
    with open(filename, 'wb') as f:
        f.write(response.content)

from speisekarte_pizza_sample import MENU

SyntaxError: invalid syntax (2434350613.py, line 19)

In [ ]:
from openai import OpenAI
from google.colab import userdata

openai_api_key = userdata.get('OPENAI_API_KEY')
openai_client = OpenAI(api_key = openai_api_key)

In [ ]:
from typing import List, Dict


# Bestellung (als globale Liste)
bestellung: List[Dict] = []


def add_item_to_order(item_id: str) -> str:
    """
    Python-Funktion, welche ein Item zur Bestellung hinzufügt. Dabei wird vorab geprüft, ob das Item in der
    Speisekarte vorhanden ist. Falls nicht, wird eine Fehlermeldung auf der Konsole ausgegeben.

    Args:
        item_id (str): Eindeutige ID des Items, welches zur Bestellung hinzugefügt werden soll.

    Returns:
        str: Text, der angibt, ob das Item erfolgreich zur Bestellung hinzugefügt wurde oder ob ein Fehler aufgetreten ist.
    """
    item = next((entry for entry in MENU if entry["id"] == item_id), None)
    if item:
        bestellung.append(item)
        return f"{item['name']} wurde zur Bestellung hinzugefügt."
    return f"Item mit ID {item_id} nicht gefunden."

Im folgenden Abschnitt werden alle verfügbaren Tools aufgelistet, die in der OpenAI API verwendet werden können. Diese Tools sind in der Regel Funktionen, die aufgerufen werden können, um bestimmte Aufgaben zu erledigen, wie z.B. das Abrufen von Informationen oder das Ausführen von Berechnungen.

In [ ]:
available_tools = [
    {
        "type": "function",
        "name": "add_item_to_order",
        "description": "Fügt ein Item der Bestellung hinzu.",
        "parameters": {
            "type": "object",
            "properties": {
                "item_id": {
                    "type": "string",
                    "description": "Die ID des hinzuzufügenden Items.",
                }
            },
            "required": ["item_id"],
        },
    },
]

In [ ]:
import json

def send_prompt(user_input: str) -> Dict:
	"""
	Sendet eine Anfrage an das OpenAI-Modell mit der Benutzeranfrage und den verfügbaren Tools.

	Args:
		user_input (str): Die Eingabe des Benutzers, die verarbeitet werden soll.

	Returns:
		Dict: Die Antwort des Modells, die entweder eine Textantwort oder einen Funktionsaufruf enthält.
	"""
	response = openai_client.responses.create(
		model = "gpt-4o",
		input = [
			{
				"role": "developer",
				"content": "Du bist ein digitaler Kellner. Nimm Bestellungen auf und beantworte Fragen zum Menü."
			},
			{
				"role": "developer", # dieses mal nicht als Prompt injection, sondern als Kontext
				"content": f"Speisekarte:\n{json.dumps(MENU, ensure_ascii=False)}",
			},
			{
				"role": "user",
				"content": user_input,
			},
		],
        tools = available_tools,
        tool_choice = "auto",
	)

	for tool_call in response.output:
		if tool_call.type != "function_call":
			print(f"Antwort: {response.output_text}")
			continue

		name = tool_call.name
		args = json.loads(tool_call.arguments)

		if name == "add_item_to_order":
			result = add_item_to_order(args["item_id"])
		print(f"Function Call: {name}({args}) ➜ {result}")

		# Anzeige der aktuellen Bestellung
		print("📦 Aktuelle Bestellung:")
		for item in bestellung:
			print(f"- {item['name']} ({item['preis']} €)")
		print('-' * 40+'\n')

In [ ]:
send_prompt("Wie geht es dir? ")
send_prompt("Ich möchte gerne eine Pizza Diavola und eine Cola bestellen.")
send_prompt("Noch eine Cola bitte")
send_prompt("Bitte entferne die Cola wieder.")

**Aufgabe**: Erweitere die Funktionalität um eine Funktion "remove_item_from_order", welche die id eines bestelltem Items übergeben bekommt und dieses aus der Liste löscht.